In [1]:
import re
import urllib
import json
import numpy as np
import pandas as pd
import transformers
import tensorflow as tf
import torch.nn as nn
import texthero as hero
from urllib.parse import urlsplit
import requests
import torch
from bs4 import BeautifulSoup
from requests import get
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from huggingface_hub import from_pretrained_keras
import spacy_sentence_bert

In [2]:
from summarizer import Summarizer,TransformerSummarizer

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from timeit import default_timer as timer

### TO DO
1. News section search - Done
2. Abstarctive summary
3. Specify cap on number of links
4. Search Keywords
5. Tune similarity parameter n Model
6. Use summary or not

## Search Google News

In [5]:
import math
from GoogleNews import GoogleNews
from datetime import datetime

In [6]:
googlenews = GoogleNews(lang='en')
googlenews.set_encode('utf-8')

In [7]:
def get_links_from_google_news(claim, top=10):
    googlenews.search(claim)
    
    results = googlenews.results()
    googlenews.clear()
    
    results = [x for x in results if isinstance(x['datetime'], datetime)]
    
    def func(ele):
        return str(ele['datetime'])
    
    results.sort(key = func, reverse = True)
    links = [x['link'] for x in results]
    
    return links

## Scraping Evidences

In [8]:
nlp = spacy_sentence_bert.load_model('en_nli_roberta_base')

In [9]:
def get_sentences_from_link(link, text, top=10):
    request = requests.get(link, verify=False, timeout=20)
    Soup = BeautifulSoup(request.text, 'lxml')
    
    heading_tags = ['p']


    results = []
    used = []

    for tags in Soup.find_all(heading_tags):
        if 'h' in tags.name:
            tokens = tags.text.strip().split()
            if len(tokens) > 8:
                if tags.text.strip() not in used:
                    used.append(tags.text.strip())
                    results.append([tags.name, tags.text.strip()])
        else:
            tokens = tags.text.strip().split()
            if len(tokens) > 8:
                if tags.text.strip() not in used:
                    used.append(tags.text.strip())
                    results.append([tags.name, tags.text.strip()])
    doc1 = nlp(text)
    sim = []
    for r in results:
        sim.append(doc1.similarity(nlp(r[1])))
    zipped = zip(sim, results)
    zipped = sorted(zipped, reverse=True)
    high_conf = [a for s, a in zipped if s >= 0.6]

    return high_conf[:top], request.url

In [10]:
def scrap_evidences(text, links):
    new_links = []
    for link in links:
        conf, lin = get_sentences_from_link(link, text)
        new_links.append([lin, conf])
    return new_links

## Concatinating Evidences

In [11]:
def concatenate_evidences(claim, links):
    summ = []
    for link in links:
        if type(link[1]) == list:
            for text in link[1]:
                if type(link[1]) == list:
                    summ.append(text[1])
                else:
                    summ.append(text)
        elif type(link[1]) == str:
            summ.append(link[1])

    urls = re.findall(r'https?:\/+\/+t+\.+co+\/+\S*', claim)
    
    for li in urls:
        claim = claim.replace(li, '')
    claim = claim.strip()

    if summ:
        summary = (claim, ' '.join(summ).replace('\n', '').replace('\t', ''))
    else:
        summary = ('', '')

    return summary

## Extractive Summary

In [12]:
summarizer = Summarizer()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
def generate_extractive_summary(evidence):
    return ''.join(summarizer(evidence, min_length=60))

## Abstractive Summary

In [14]:
from transformers import pipeline
import os

In [15]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [16]:
abstractive_summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [133]:
abstractive_summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

2023-04-11 11:48:42.879 INFO    filelock: Lock 140411692340608 acquired on /Users/shivamarora/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b.lock


2023-04-11 11:48:43.468 INFO    filelock: Lock 140411692340608 released on /Users/shivamarora/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.a085189636e38420df5e7bdf08ad1b86f1fe33c010079ca7f15437ff95f4fe2b.lock


2023-04-11 11:48:45.418 INFO    filelock: Lock 140409722690288 acquired on /Users/shivamarora/.cache/huggingface/transformers/748a176e9d151dcad63a27974db8b8f665f286954cfbb77008ca42163419ff66.6a323429db2b09562cffdb9bc72d09d08bccbca1d832434b183b867864c30526.h5.lock


2023-04-11 11:52:22.034 INFO    filelock: Lock 140409722690288 released on /Users/shivamarora/.cache/huggingface/transformers/748a176e9d151dcad63a27974db8b8f665f286954cfbb77008ca42163419ff66.6a323429db2b09562cffdb9bc72d09d08bccbca1d832434b183b867864c30526.h5.lock


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
2023-04-11 11:52:31.487 INFO    filelock: Lock 140413416447712 acquired on /Users/shivamarora/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock


2023-04-11 11:52:32.232 INFO    filelock: Lock 140413416447712 released on /Users/shivamarora/.cache/huggingface/transformers/684a47ca6257e4ca71f0037771464c5b323e945fbc58697d2fad8a7dd1a2f8ba.3b69006860e7b5d0a63ffdddc01ddcd6b7c318a6f4fd793596552c741734c62d.lock


2023-04-11 11:52:32.501 INFO    filelock: Lock 140413452874656 acquired on /Users/shivamarora/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock


2023-04-11 11:52:33.343 INFO    filelock: Lock 140413452874656 released on /Users/shivamarora/.cache/huggingface/transformers/90de37880b5ff5ac7ab70ff0bd369f207e9b74133fa153c163d14c5bb0116207.8627f1bd5d270a9fd2e5a51c8bec3223896587cc3cfe13edeabb0992ab43c529.lock


In [17]:
def generate_abstractive_summary(evidence):
    return abstractive_summarizer(evidence, min_length=5, do_sample=False)[0]['summary_text']

## Semantic Based Similarity

In [18]:
class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data."""
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=32,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

In [19]:
model = from_pretrained_keras("keras-io/bert-semantic-similarity")
labels = ["contradiction", "entailment", "neutral"]

config.json not found in HuggingFace Hub


In [20]:
def predict(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    probs = model.predict(test_data[0])[0]
    
    labels_probs = {labels[i]: float(probs[i]) for i, _ in enumerate(labels)}
    return labels_probs

## Fake News Detection

In [143]:
def fake_news_detection(claim):
    start = timer()
    links = get_links_from_google_news(claim)
#     print("links ----- ")
#     print(links)
    evidence_list = scrap_evidences(claim, links)
#     print("Evidence List ----- ")
#     print(evidence_list)
    evidence = concatenate_evidences(claim, evidence_list)
#     print("Concatenated Evidence ----- ")
#     print(evidence[1])
    print("Results with Concatenated Evidence")
    print(predict(claim, evidence[1]))
    extractive_summary = generate_extractive_summary(evidence[1])
    print("Extractive Summary ----- ")
    print(extractive_summary)
    print("Results with Extractive Summary")
    print(predict(claim, extractive_summary))
#     abstractive_summary = generate_abstractive_summary(evidence[1])
#     print("Abstractive Summary ----- ")
#     print(abstractive_summary)
#     print("Results with Abstractive Summary")
#     print(predict(claim, abstractive_summary))
    end = timer()
    print(end - start)
#     return evidence[1]

    

In [145]:
claim = "Joe biden announces provision of free meals in every school"
# claim = "People from LGBTQ Community can marry in india"
v = fake_news_detection(claim)


Results with Concatenated Evidence


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


1/1 [==============================] - 0s 480ms/step
{'contradiction': 0.8693470358848572, 'entailment': 0.0003093259874731302, 'neutral': 0.13034358620643616}
Extractive Summary ----- 
The mother of three, attuned to the needs of the area’s children, identified other ways to support them. Students from every Hartford-area school have come through her program. President Biden will first travel to Belfast, Northern Ireland, from April 11-12 to mark the tremendous progress since the signing of the Belfast/Good Friday Agreement 25 years ago and to underscore the readiness of the United States to support Northern Ireland’s vast economic potential to the benefit of all communities. On March 27, the Senate narrowly rejected a bill, which had passed the House, guaranteeing free school lunches to K-12 students in families at or below 200 percent of the federal poverty level. Universal preschool and affordable child care: The budget would provide funding for a new federal-state partnership prog

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


1/1 [==============================] - 0s 227ms/step
{'contradiction': 0.12236171215772629, 'entailment': 3.284166086814366e-05, 'neutral': 0.8776054382324219}
200.3807900000029


In [138]:
print(v)

RSS is an organization that supports Indian culture and values. OAS commission urges Panama to extend marriage rights to same-sex couples Sadam Hanjabam, founder of Ya_all, an LGBTQ and intersex rights organization based in Manipur, told the Blade the government still looks at the family from the angle of male and female. Bobby Kinner is a member of the Aam Aadmi Party “As a community-based organization, our job is to prepare the community on the ground for such legal proceedings. We have so many beautifully written judgments in past by the Supreme Court of India, but when it comes to implementation, that’s the challenge,” said Chopade. “We can prepare the community for the responses or implementation. For example, if tomorrow we have marriage equality, then as an LGBTQ organization, our job is to disperse the judgment.” A spokesperson of the Sappho for Equality, an LGBTQ and intersex organization based in the Indian state of West Bengal, told the Blade that equality must extend to all

In [139]:
generate_abstractive_summary(v)

"india's top court has been supportive of gay rights . a pivotal case currently being considered in the country could lead to the legalization of same-sex marriages . the law minister said that marriage is an institution guided by laws enacted by Parliament ."